In [63]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer

In [64]:
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\joaov\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [65]:
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'\W', ' ', text)
    text = re.sub(r"\d+", '', text)
    text = ' '.join([word for word in text.split() if ((word not in stop_words) and (len(word) >= 3))])
    return text

In [66]:
df = pd.read_csv('C:\\Users\\joaov\\Documents\\Whisky Recomendation System\\Whisky Notes Extraction\\Step 2\\Whisky Notes - Step 2.csv', sep=';')
df = df.drop_duplicates().reset_index(drop=True)
df

,review_date,review_title,review_link,review_full_text,review_image,review_score
0,2025-10-03,"A Secret Highland Distillery 14 yo 2010 (54,1%...",https://www.whiskynotes.be/2025/caol-ila/whisk...,"Nose: a lot of sherry influence, but never in-...",https://www.whiskynotes.be/wp-content/uploads/...,84
1,2025-10-03,"Inchgower 13 yo 2012 (56,4%, Whiskydudes 2015,...",https://www.whiskynotes.be/2025/caol-ila/whisk...,"Nose: lots of butter biscuits at first, even a...",https://www.whiskynotes.be/wp-content/uploads/...,86
2,2025-10-03,"Secret Campbeltown Blended Malt 8 yo 2016 (58,...",https://www.whiskynotes.be/2025/caol-ila/whisk...,"Nose: nice gristy notes, some balanced maritim...",https://www.whiskynotes.be/wp-content/uploads/...,88
3,2025-10-03,"Caol Ila 9 yo 2014 (57,6%, Whiskydudes 2024, P...",https://www.whiskynotes.be/2025/caol-ila/whisk...,"Nose: rich and full, albeit with a surprisingl...",https://www.whiskynotes.be/wp-content/uploads/...,88
4,2025-10-03,"Caol Ila 10 yo 2015 (55,6%, Whiskydudes 2025, ...",https://www.whiskynotes.be/2025/caol-ila/whisk...,Nose: sour wine and sour berries come out firs...,https://www.whiskynotes.be/wp-content/uploads/...,84
...,...,...,...,...,...,...
5824,2008-12-11,"Clynelish 12y (57%, G&M 1980’s – Ainslie & Hei...",https://www.whiskynotes.be/2008/clynelish/clyn...,"Nose: dried fruits (plums, raisins). Pine fore...",https://www.whiskynotes.be/upload/Clynelish12y...,93
5825,2008-12-10,"Banff 25y 1980 (56,5%, DT Rarest of the Rare 2...",https://www.whiskynotes.be/2008/banff/banff-25...,"Apricot and frangipane on the nose. Cake, vani...",https://www.whiskynotes.be/upload/Banff25y1980...,84
5826,2008-12-09,"Teaninich 23y 1972 (64,95%, Rare Malts 1996)",https://www.whiskynotes.be/2008/teaninich/tean...,"Interesting nose: wood, crème de banane, beesw...",https://www.whiskynotes.be/upload/Teaninich23y...,90
5827,2008-12-08,"Craigduff 32y 1973 (49,4%, SV 2005, cask #2513...",https://www.whiskynotes.be/2008/glen-keith/cra...,"Beautiful nose, with notes of marzipan, ginger...",https://www.whiskynotes.be/upload/Craigduff32y...,86


In [67]:
df['review_score'] = df['review_score'].str.replace('+', '').str.replace('.', '')
df = df[~((df['review_score'].isna()) | (df['review_score'] == ''))]
df['review_score'] = df['review_score'].astype(int, errors='ignore')
df = df[(df['review_score'] >= 80) & (df['review_score'] <= 100)]
df

C:\Users\joaov\AppData\Local\Temp\ipykernel_15200\856402699.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['review_score'] = df['review_score'].astype(int, errors='ignore')


,review_date,review_title,review_link,review_full_text,review_image,review_score
0,2025-10-03,"A Secret Highland Distillery 14 yo 2010 (54,1%...",https://www.whiskynotes.be/2025/caol-ila/whisk...,"Nose: a lot of sherry influence, but never in-...",https://www.whiskynotes.be/wp-content/uploads/...,84
1,2025-10-03,"Inchgower 13 yo 2012 (56,4%, Whiskydudes 2015,...",https://www.whiskynotes.be/2025/caol-ila/whisk...,"Nose: lots of butter biscuits at first, even a...",https://www.whiskynotes.be/wp-content/uploads/...,86
2,2025-10-03,"Secret Campbeltown Blended Malt 8 yo 2016 (58,...",https://www.whiskynotes.be/2025/caol-ila/whisk...,"Nose: nice gristy notes, some balanced maritim...",https://www.whiskynotes.be/wp-content/uploads/...,88
3,2025-10-03,"Caol Ila 9 yo 2014 (57,6%, Whiskydudes 2024, P...",https://www.whiskynotes.be/2025/caol-ila/whisk...,"Nose: rich and full, albeit with a surprisingl...",https://www.whiskynotes.be/wp-content/uploads/...,88
4,2025-10-03,"Caol Ila 10 yo 2015 (55,6%, Whiskydudes 2025, ...",https://www.whiskynotes.be/2025/caol-ila/whisk...,Nose: sour wine and sour berries come out firs...,https://www.whiskynotes.be/wp-content/uploads/...,84
...,...,...,...,...,...,...
5824,2008-12-11,"Clynelish 12y (57%, G&M 1980’s – Ainslie & Hei...",https://www.whiskynotes.be/2008/clynelish/clyn...,"Nose: dried fruits (plums, raisins). Pine fore...",https://www.whiskynotes.be/upload/Clynelish12y...,93
5825,2008-12-10,"Banff 25y 1980 (56,5%, DT Rarest of the Rare 2...",https://www.whiskynotes.be/2008/banff/banff-25...,"Apricot and frangipane on the nose. Cake, vani...",https://www.whiskynotes.be/upload/Banff25y1980...,84
5826,2008-12-09,"Teaninich 23y 1972 (64,95%, Rare Malts 1996)",https://www.whiskynotes.be/2008/teaninich/tean...,"Interesting nose: wood, crème de banane, beesw...",https://www.whiskynotes.be/upload/Teaninich23y...,90
5827,2008-12-08,"Craigduff 32y 1973 (49,4%, SV 2005, cask #2513...",https://www.whiskynotes.be/2008/glen-keith/cra...,"Beautiful nose, with notes of marzipan, ginger...",https://www.whiskynotes.be/upload/Craigduff32y...,86


In [68]:
df = df[['review_title', 'review_full_text']]

In [69]:
#df['review_title'] = df['review_title'].str.lower().str.replace('-', ' ')
df['review_full_text'] = df['review_full_text'].str.lower().str.replace('-', ' ')

C:\Users\joaov\AppData\Local\Temp\ipykernel_15200\3665824303.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['review_full_text'] = df['review_full_text'].str.lower().str.replace('-', ' ')


## For future usage

In [70]:
# df['age'] = df['review_title'].str.extract(r'(\d+)\s*yo', flags=re.I).astype('Int64')
# df['abv'] = df['review_title'].str.extract(r'(\d+[,.]?\d*)\s*%', flags=re.I)
# df['abv'] = df['abv'].replace(',', '.', regex=True).astype(float)


In [71]:
# # For future usage

# df['fill'] = np.where((df['review_title'].str.contains('first fill|fresh fill', case=False, na=False)), 'First Fill',
#              np.where((df['review_title'].str.contains('second fill|refill', case=False, na=False)), 'Second Fill',
#              np.where((df['review_title'].str.contains('third fill', case=False, na=False)), 'Third Fill',
#              None)))
                    
# df['previous_spirit'] = np.where((df['review_title'].str.contains('sherry|jerez|xerez|pedro ximenez|px|oloroso|fino|amontillado|palo cortado', case=False, na=False)), 'Sherry',
#                         np.where((df['review_title'].str.contains('bourbon', case=False, na=False)), 'Bourbon',
#                         np.where((df['review_title'].str.contains('madeira', case=False, na=False)), 'Madeira',
#                         np.where((df['review_title'].str.contains('port', case=False, na=False)), 'Port',
#                         np.where((df['review_title'].str.contains('wine', case=False, na=False)), 'Wine',
#                         np.where((df['review_title'].str.contains('cognac|armagnac', case=False, na=False)), 'Cognac/Armagnac',
#                         np.where((df['review_title'].str.contains('rum', case=False, na=False)), 'Rum',
#                         np.where((df['review_title'].str.contains('virgin', case=False, na=False)), 'Virgin Oak',
#                         np.where((df['review_title'].str.contains('amarone', case=False, na=False)), 'Amarone',
#                         None)))))))))

In [72]:
# dim_distilleries = pd.read_csv('dim_distilleries.csv', sep=';')
# dim_distilleries

In [73]:
# dim_distilleries = dim_distilleries[dim_distilleries['Is Closed'] == False].reset_index(drop=True)
# dim_distilleries.columns = dim_distilleries.columns.str.strip().str.replace(' ', '_').str.lower()
# dim_distilleries = dim_distilleries[['distillery', 'region', 'type_of_whisky','latitude','longitude']]
# dim_distilleries['distillery'] = dim_distilleries['distillery'].str.lower()
# dim_distilleries

In [74]:
# distilleries_list = dim_distilleries['distillery'].tolist()
# df['distillery'] = np.nan

# for distillery in distilleries_list:
#     df.loc[df["review_title"].str.contains(fr'(^|[\s,;.!?]){distillery}($|[\s,;.!?])', case=False, na=False), 'distillery'] = distillery

In [75]:
# df = df.merge(dim_distilleries, on='distillery', how='left')

In [76]:
# df['type_of_spirit'] = np.where(df['review_title'].str.contains('cognac', na=False), 'Cognac',
#                        np.where(df['review_title'].str.contains('armagnac', na=False), 'Armagnac',
#                        np.where(df['review_title'].str.contains('rum', na=False), 'Rum',
#                        np.where(df['review_title'].str.contains('eau-de-vie', na=False), 'Eau-de-Vie',
#                        np.where(df['review_title'].str.contains('new make', na=False), 'New Make',
#                        'Whisky')))))

In [77]:
# df['cask_size'] = np.where(df['review_title'].str.contains('hogshead', na=False), 'Hogshead',
#                   np.where(df['review_title'].str.contains('barrel', na=False), 'Barrel',
#                   np.where(df['review_title'].str.contains('butt', na=False), 'Butt',
#                   np.where(df['review_title'].str.contains('puncheon', na=False), 'Puncheon',
#                   np.where(df['review_title'].str.contains('quarter cask', na=False), 'Quarter Cask',
#                   np.where(df['review_title'].str.contains('octave', na=False), 'Octave',
#                   np.where(df['review_title'].str.contains('cask', na=False), 'Cask',
#                   None)))))))

In [78]:
# df['cask_fill'] = np.where(df['review_title'].str.replace('-', ' ').str.contains('first fill', na=False), 'First Fill',
#                   np.where(df['review_title'].str.replace('-', ' ').str.contains('fresh fill', na=False), 'First Fill',
#                   np.where(df['review_title'].str.replace('-', ' ').str.contains('second fill', na=False), 'Second Fill',
#                   np.where(df['review_title'].str.replace('-', ' ').str.contains('refill', na=False), 'Second Fill',
#                   np.where(df['review_title'].str.replace('-', ' ').str.contains('third fill', na=False), 'Third Fill',
#                   None)))))

In [79]:
# df['previous_spirit'] = np.where(df['review_title'].str.replace('-', ' ').str.contains('bourbon', na=False), 'Bourbon',
#                         np.where(df['review_title'].str.replace('-', ' ').str.contains('sherry', na=False), 'Sherry',
#                         np.where(df['review_title'].str.replace('-', ' ').str.contains('oloroso', na=False), 'Sherry',
#                         np.where(df['review_title'].str.replace('-', ' ').str.contains('px', na=False), 'Sherry',
#                         np.where(df['review_title'].str.replace('-', ' ').str.contains('pedro ximénez', na=False), 'Sherry',
#                         np.where(df['review_title'].str.replace('-', ' ').str.contains('palo cortado', na=False), 'Sherry',
#                         np.where(df['review_title'].str.replace('-', ' ').str.contains('amontillado', na=False), 'Sherry',
#                         np.where(df['review_title'].str.replace('-', ' ').str.contains('fino', na=False), 'Sherry',
#                         np.where(df['review_title'].str.replace('-', ' ').str.contains('wine', na=False), 'Wine',
#                         np.where(df['review_title'].str.replace('-', ' ').str.contains('amarone', na=False), 'Wine',
#                         np.where(df['review_title'].str.replace('-', ' ').str.contains('rivesaltes', na=False), 'Wine',
#                         np.where(df['review_title'].str.replace('-', ' ').str.contains('rum', na=False) & (df['type_of_spirit'] != 'Rum'), 'Rum',
#                         np.where(df['review_title'].str.replace('-', ' ').str.contains('cognac', na=False) & (df['type_of_spirit'] != 'Cognac'), 'Cognac',
#                         np.where(df['review_title'].str.replace('-', ' ').str.contains('armagnac', na=False) & (df['type_of_spirit'] != 'Armagnac'), 'Armagnac',
#                         np.where(df['review_title'].str.replace('-', ' ').str.contains('port', na=False), 'Port',
#                         np.where(df['review_title'].str.replace('-', ' ').str.contains('tequila', na=False), 'Tequila',
#                         np.where(df['review_title'].str.replace('-', ' ').str.contains('madeira', na=False), 'Madeira',
#                         np.where(df['review_title'].str.replace('-', ' ').str.contains('vermouth', na=False), 'Vermouth',
#                         np.where(df['review_title'].str.replace('-', ' ').str.contains('virgin oak', na=False), 'Virgin Oak',
#                         None)))))))))))))))))))

## -----

In [80]:
df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)

C:\Users\joaov\AppData\Local\Temp\ipykernel_15200\3528310598.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(inplace=True)


In [81]:
df.loc[:, 'review_full_text'] = df['review_full_text'].str.replace('nose:|mouth:|finish:', '', regex=True)

In [82]:
parser_porter_stemmer = PorterStemmer()

def stemmer(text):

    y = []

    text_words = text.split()

    for word in text_words:
        y.append(parser_porter_stemmer.stem(word))
    
    stemmed_text = ' '.join(y)

    return stemmed_text

# parser_sb_stemmer = SnowballStemmer(language='english')

# def stemmer(text):

#     text_words = text

#     groups = {}

#     for word in text_words:

#         root = parser_sb_stemmer.stem(word)

#         if root not in groups:
#             groups[root] = []

#         groups[root].append(word)
    
#     clean_words = [min(group, key=len) for group in groups.values()]

#     clean_words = sorted(set(clean_words))

#     stemmed_text = ' '.join(clean_words)

#     return stemmed_text

In [83]:
df.loc[:, 'review_full_text_processed'] = [preprocess_text(text) for text in df['review_full_text']]

C:\Users\joaov\AppData\Local\Temp\ipykernel_15200\2149076747.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, 'review_full_text_processed'] = [preprocess_text(text) for text in df['review_full_text']]


In [84]:
df.loc[:, 'review_full_text_processed'] = df['review_full_text_processed'].apply(stemmer)

In [85]:
normalize_map = {
    "add": "add",
    "almond": "almond",
    "alcohol": "alcohol",
    "appear": "appear",
    "appl": "apple",
    "apricot": "apricot",
    "ash": "ash",
    "banana": "banana",
    "berri": "berry",
    "best": "best",
    "better": "better",
    "blackberri": "blackberry",
    "bring": "bring",
    "butter": "butter",
    "butteri": "butter",
    "candi": "candy",
    "caramel": "caramel",
    "cask": "cask",
    'chalki': 'chalk',
    "char": "char",
    "chocol": "chocolate",
    "cigar": "cigar",
    "cinnamon": "cinnamon",
    "citru": "citrus",
    "citrusi": "citrus",
    'cherri': 'cherry',
    "clove": "clove",
    "coffe": "coffee",
    "coffee": "coffee",
    "come": "come",
    "cream": "cream",
    "creami": "cream",
    "dark": "dark",
    "darker": "dark",
    "distilleri": "distillery",
    "dri": "dry",
    "drier": "dry",
    "dry": "dry",
    'dusti': 'dust',
    'earthi': 'earthy',
    "eucalyptu": "eucalyptus",
    "fade": "fade",
    "find": "find",
    "finish": "finish",
    "flower": "floral",
    "floral": "floral",
    "fragrant": "aroma",
    "fruit": "fruit",
    "fruiti": "fruit",
    "full": "full",
    "get": "get",
    "gingeri": "ginger",
    "gooseberri": "gooseberry",
    "good": "good",
    "grain": "grain",
    "graini": "grain",
    "grape": "grape",
    "grapefruit": "grapefruit",
    "grass": "grass",
    "grassi": "grass",
    "great": "great",
    "grow": "grow",
    "hazelnut": "hazelnut",
    "heather": "heather",
    "herb": "herb",
    "herbal": "herb",
    "honey": "honey",
    "intens": "intense",
    "iodin": "iodine",
    'leafi': 'leaf',
    'leatheri': 'leather',
    "length": "length",
    'liquoric': 'liqueur',
    "light": "light",
    "lightli": "light",
    "linger": "linger",
    "liqueur": "liqueur",
    "long": "long",
    "juici": "juicy",
    "juic": "juicy",
    "make": "make",
    "malt": "malt",
    "malti": "malt",
    "mango": "mango",
    "marzipan": "marzipan",
    "milk": "milk",
    'miner': 'mineral',
    "mint": "mint",
    "minti": "mint",
    "move": "move",
    "nut": "nut",
    "nutmeg": "nutmeg",
    "nutti": "nut",
    "oak": "oak",
    "oaki": "oak",
    "oil": "oily",
    "oily": "oily",
    "oili": "oily",
    "orange": "orange",
    "orang": "orange",
    'passion': 'passion fruit',
    "pastri": "pastry",
    "peach": "peach",
    "pear": "pear",
    "peat": "peat",
    "peati": "peat",
    "pepper": "pepper",
    "pepperi": "pepper",
    "peppercorn": "peppercorn",
    "pineappl": "pineapple",
    "pink": "pink",
    "plum": "plum",
    "raspberri": "raspberry",
    "red": "red",
    "resin": "resin",
    "rum": "rum",
    "round": "round",
    'salti': 'salt',
    "say": "say",
    "seem": "seem",
    "shake": "smoke",
    "sherri": "sherry",
    "sherry": "sherry",
    "show": "show",
    "smell": "aroma",
    "smoke": "smoke",
    "smoki": "smoke",
    "smooth": "smooth",
    "soft": "soft",
    "spice": "spice",
    "spici": "spice",
    "spirit": "spirit",
    "strength": "strength",
    "strong": "strong",
    "sugar": "sugar",
    "sweet": "sweet",
    "sweeter": "sweet",
    "syrup": "syrup",
    "take": "take",
    "tea": "tea",
    "toast": "toast",
    "toffe": "toffee",
    "tobacco": "tobacco",
    "tri": "try",
    "try": "try",
    "vanilla": "vanilla",
    "wax": "wax",
    "waxi": "wax",
    "white": "white",
    "whiski": "whisky",
    "wine": "wine",
    "wood": "wood",
    "woodi": "wood",
    "yellow": "yellow",
    "zest": "zest",
    "zesti": "zest"
}

def normalize_words(phrase):
    phrase = phrase.split()
    normalized_phrase = [normalize_map.get(word, word) for word in phrase]
    return " ".join(normalized_phrase)

df["review_full_text_processed"] = df["review_full_text_processed"].apply(normalize_words)

C:\Users\joaov\AppData\Local\Temp\ipykernel_15200\695551549.py:165: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["review_full_text_processed"] = df["review_full_text_processed"].apply(normalize_words)


In [86]:
counter = CountVectorizer(max_df=0.8, min_df=0.05, strip_accents='unicode', binary=True)
counter

,input,'content'
,encoding,'utf-8'
,decode_error,'strict'
,strip_accents,'unicode'
,lowercase,True
,preprocessor,None
,tokenizer,None
,stop_words,None
,token_pattern,'(?u)\\b\\w\\w+\\b'
,ngram_range,"(1, ...)"
,analyzer,'word'


In [87]:
counter.fit_transform(df['review_full_text_processed']).toarray()

array([[0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], shape=(5358, 290))

In [88]:
reference_words = counter.get_feature_names_out()
reference_words

array(['add', 'age', 'alcohol', 'almond', 'almost', 'along', 'alongsid',
       'also', 'although', 'alway', 'anise', 'appear', 'apple', 'apricot',
       'aroma', 'aromat', 'around', 'ash', 'ashi', 'auster', 'avail',
       'back', 'background', 'bad', 'bake', 'balanc', 'banana', 'barley',
       'beauti', 'becom', 'berry', 'best', 'better', 'big', 'bit',
       'bitter', 'black', 'blackberry', 'bottl', 'bourbon', 'bread',
       'bright', 'brine', 'bring', 'brown', 'burnt', 'butter', 'cake',
       'candy', 'caramel', 'cask', 'certainli', 'chalk', 'char',
       'charact', 'cherry', 'chocolate', 'cigar', 'cinnamon', 'citrus',
       'classic', 'clean', 'close', 'clove', 'coastal', 'cocoa',
       'coconut', 'coffee', 'combin', 'come', 'complex', 'could', 'cours',
       'cream', 'custard', 'dark', 'definit', 'differ', 'distillery',
       'dram', 'drop', 'dry', 'dust', 'earthy', 'easi', 'edg', 'eleg',
       'element', 'end', 'eucalyptus', 'even', 'excel', 'exchang',
       'expect',

In [89]:
tasting_notes_words_filtered = [
    'almond',
    'anise',
    'apple',
    'apricot',
    'ash',
    'banana',
    'barley',
    'berry',
    'bitter',
    'blackberry',
    'bread',
    'brine',
    'burnt',
    'butter',
    'cake',
    'candy',
    'caramel',
    'chalk',
    'char',
    'cherry',
    'chocolate',
    'cigar',
    'cinnamon',
    'citrus',
    'clove',
    'cocoa',
    'coconut',
    'coffee',
    'cream',
    'custard',
    'dust',
    'earthy',
    'eucalyptus',
    'fig',
    'floral',
    'fresh',
    'fruit',
    'ginger',
    'gooseberry',
    'grain',
    'grape',
    'grapefruit',
    'grass',
    'green',
    'hay',
    'hazelnut',
    'heather',
    'herb',
    'honey',
    'iodine',
    'jam',
    'juicy',
    'leather',
    'lemon',
    'lime',
    'liqueur',
    'malt',
    'mango',
    'marmalad',
    'marzipan',
    'medicin',
    'melon',
    'menthol',
    'milk',
    'mineral',
    'mint',
    'mocha',
    'nut',
    'nutmeg',
    'oak',
    'oily',
    'orange',
    'passion fruit',
    'pastry',
    'peach',
    'pear',
    'peat',
    'peel',
    'pepper',
    'peppercorn',
    'pine',
    'pineapple',
    'plum',
    'raisin',
    'raspberry',
    'resin',
    'ripe',
    'roast',
    'rum',
    'salt',
    'sherry',
    'smoke',
    'sour',
    'spice',
    'stew',
    'sugar',
    'sweet',
    'syrup',
    'tangerin',
    'tea',
    'toast',
    'tobacco',
    'toffee',
    'tropic',
    'vanilla',
    'varnish',
    'walnut',
    'warm',
    'wax',
    'wine',
    'wood',
    'zest'
]

In [90]:
counter = CountVectorizer(vocabulary=tasting_notes_words_filtered, max_df=0.8, min_df=0.05, strip_accents='unicode', binary=True)
counter

,input,'content'
,encoding,'utf-8'
,decode_error,'strict'
,strip_accents,'unicode'
,lowercase,True
,preprocessor,None
,tokenizer,None
,stop_words,None
,token_pattern,'(?u)\\b\\w\\w+\\b'
,ngram_range,"(1, ...)"
,analyzer,'word'


In [91]:
review_text_vec = counter.fit_transform(df['review_full_text_processed']).toarray()
review_text_vec

array([[0, 0, 1, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 1, 0],
       [0, 0, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], shape=(5358, 112))

In [92]:
df['review_text_vector'] = review_text_vec.tolist()

C:\Users\joaov\AppData\Local\Temp\ipykernel_15200\2636747614.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['review_text_vector'] = review_text_vec.tolist()


In [93]:
df = df[['review_title', 'review_text_vector']]

In [94]:
df

,review_title,review_text_vector
0,"A Secret Highland Distillery 14 yo 2010 (54,1%...","[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, ..."
1,"Inchgower 13 yo 2012 (56,4%, Whiskydudes 2015,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, ..."
2,"Secret Campbeltown Blended Malt 8 yo 2016 (58,...","[0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,"Caol Ila 9 yo 2014 (57,6%, Whiskydudes 2024, P...","[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ..."
4,"Caol Ila 10 yo 2015 (55,6%, Whiskydudes 2025, ...","[0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...
5353,"Clynelish 12y (57%, G&M 1980’s – Ainslie & Hei...","[0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, ..."
5354,"Banff 25y 1980 (56,5%, DT Rarest of the Rare 2...","[0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, ..."
5355,"Teaninich 23y 1972 (64,95%, Rare Malts 1996)","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
5356,"Craigduff 32y 1973 (49,4%, SV 2005, cask #2513...","[0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, ..."


In [95]:
df.to_csv('reviews_vectors.csv', index=False)

In [96]:
df_tasting_notes = []

for i in df.index:

    review_title = df.loc[i, 'review_title']

    for j in range(len(tasting_notes_words_filtered)):

        tasting_note = tasting_notes_words_filtered[j]

        df_tasting_note = {
            'review_title': review_title,
            'tasting_note': tasting_note,
            'has_tasting_note': df.loc[i, 'review_text_vector'][j]
        }

        df_tasting_notes.append(df_tasting_note)

df_tasting_notes = pd.DataFrame(df_tasting_notes)

In [97]:
tasting_notes_groups = {

    "fruity": [
        "apple", "apricot", "banana", "berry", "blackberry", "fig", "grape",
        "grapefruit", "lemon", "lime", "mango", "melon", "orange", "peach",
        "pear", "pineapple", "plum", "raisin", "jam", "marmalad",
        "gooseberry", "grass", "green", "passion fruit", "tropic",
        "citrus", "tangerin", "zest", "juicy", 'gooseberry', 'fruit',
        "ripe", "stew", "peel", 'sour', 'cherry', "raisin", "jam", 'raspberry',
        'mint', "sherry", "wine", "liqueur"
    ],

    "floral": [
        "floral", "heather", "herb", "leafy", "fresh", 'tea'
    ],

    "spicy": [
        "anise", "cinnamon", "clove", "ginger", "nutmeg", "pepper",
        "peppercorn", "warm", 'spice'
    ],

    "woody": [
        "oak", "wood", "pine", "resin", "varnish", "bitter"
    ],

    "cereal": [
        "barley", "bread", "cake", "grain", "malt", "yeast", "pastry", 'dust', "bread", 'hay'
    ],

    "peaty": [
        "peat", "smoke", "ash", "burnt", "char", "medicin", "iodine"
    ],

    "feinty": [
        "leather", "tobacco", "plastic", "oily", "wax", "mineral", "earthy", 'butter', 'chalk', 'cigar'
    ],

    "sulphury": [
        "menthol", "eucalyptus"
    ],

    "sweet": [
        "candy", "caramel", "chocolate", "cocoa", "cream", "custard",
        "honey", "sweet", "sugar", "syrup", "toffee", "rum", 'vanilla', "marmalad",
        'cake', 'milk', 'mocha'
    ],

    "nutty": [
        "almond", "hazelnut", "nut", "walnut", "marzipan", 'coconut'
    ],

    "coastal": [
        "brine", "salt", "iodine"
    ],

    "roasted": [
        "coffee", "roast", "toast"
    ]
    
}

df_tasting_notes_group = []

for tasting_notes_group in list(tasting_notes_groups.keys()):

    for tasting_note in tasting_notes_groups.get(tasting_notes_group):

        df_tasting_note_group = {
            'tasting_notes_group': tasting_notes_group,
            'tasting_note': tasting_note
        }

        df_tasting_notes_group.append(df_tasting_note_group)

df_tasting_notes_group = pd.DataFrame(df_tasting_notes_group)
df_tasting_notes_group

,tasting_notes_group,tasting_note
0,fruity,apple
1,fruity,apricot
2,fruity,banana
3,fruity,berry
4,fruity,blackberry
...,...,...
117,coastal,salt
118,coastal,iodine
119,roasted,coffee
120,roasted,roast


In [98]:
df = df.merge(df_tasting_notes, how='left', on='review_title')
df = df.merge(df_tasting_notes_group, how='left', on='tasting_note')
df

,review_title,review_text_vector,tasting_note,has_tasting_note,tasting_notes_group
0,"A Secret Highland Distillery 14 yo 2010 (54,1%...","[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, ...",almond,0,nutty
1,"A Secret Highland Distillery 14 yo 2010 (54,1%...","[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, ...",anise,0,spicy
2,"A Secret Highland Distillery 14 yo 2010 (54,1%...","[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, ...",apple,1,fruity
3,"A Secret Highland Distillery 14 yo 2010 (54,1%...","[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, ...",apricot,0,fruity
4,"A Secret Highland Distillery 14 yo 2010 (54,1%...","[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, ...",ash,0,peaty
...,...,...,...,...,...
638787,"The Balvenie 30y ‘Thirty’ (47,3%, OB 2004)","[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...",warm,0,spicy
638788,"The Balvenie 30y ‘Thirty’ (47,3%, OB 2004)","[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...",wax,0,feinty
638789,"The Balvenie 30y ‘Thirty’ (47,3%, OB 2004)","[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...",wine,0,fruity
638790,"The Balvenie 30y ‘Thirty’ (47,3%, OB 2004)","[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...",wood,0,woody


In [99]:
df_total_tasting_notes = df.groupby(['review_title']).agg(total_tasting_notes=("has_tasting_note", "sum")).reset_index()
df_total_tasting_notes

,review_title,total_tasting_notes
0,"11th Anniversary Malt\r\n(45,1%, Whisky-Doris ...",20
1,"4th Confidential Cask, 20 yo 1993 (50,3%, Kint...",24
2,"A Fine Christmas Malt 16 yo (53,2%, The Whisky...",32
3,"A Fine Christmas Malt 19 yo (44,5%, The Whisky...",32
4,"A Good Old-Fashioned Christmas Whisky (50,5%, ...",27
...,...,...
5348,"‘The Marketing Malt’ 15 yo 2008 (57,1%, Decade...",23
5349,"‘The Nameless One’ 18 yo 1995 (46,8%, The Whis...",22
5350,"‘The Trojan’ 25 yo 1990\r\n(57,1%, Exile Casks...",25
5351,"‘The Wanderer’ 24 yo 1992\r\n(54,6%, Exile Cas...",21


In [100]:
df_total_tasting_notes_group = df.groupby(['review_title', 'tasting_notes_group']).agg(total_tasting_notes_group=("has_tasting_note", "sum")).reset_index()
df_total_tasting_notes_group

,review_title,tasting_notes_group,total_tasting_notes_group
0,"11th Anniversary Malt\r\n(45,1%, Whisky-Doris ...",cereal,0
1,"11th Anniversary Malt\r\n(45,1%, Whisky-Doris ...",coastal,0
2,"11th Anniversary Malt\r\n(45,1%, Whisky-Doris ...",feinty,0
3,"11th Anniversary Malt\r\n(45,1%, Whisky-Doris ...",floral,0
4,"11th Anniversary Malt\r\n(45,1%, Whisky-Doris ...",fruity,13
...,...,...,...
64231,"“1836” 1970 (53,5%, Malts of Scotland ‘Angel’...",roasted,0
64232,"“1836” 1970 (53,5%, Malts of Scotland ‘Angel’...",spicy,1
64233,"“1836” 1970 (53,5%, Malts of Scotland ‘Angel’...",sulphury,0
64234,"“1836” 1970 (53,5%, Malts of Scotland ‘Angel’...",sweet,4


In [101]:
df_tasting_notes_group = df_total_tasting_notes_group.merge(df_total_tasting_notes, how='left', on='review_title')
df_tasting_notes_group['perc_tasting_notes_group'] = round(df_tasting_notes_group['total_tasting_notes_group'] / df_tasting_notes_group['total_tasting_notes'], 2)
df_tasting_notes_group = df_tasting_notes_group.drop(['total_tasting_notes_group', 'total_tasting_notes'], axis=1)
df_tasting_notes_group = df_tasting_notes_group[~df_tasting_notes_group['perc_tasting_notes_group'].isna()].reset_index()
df_tasting_notes_group = df_tasting_notes_group.sort_values(['review_title', 'tasting_notes_group'])
df_tasting_notes_group

,index,review_title,tasting_notes_group,perc_tasting_notes_group
0,0,"11th Anniversary Malt\r\n(45,1%, Whisky-Doris ...",cereal,0.00
1,1,"11th Anniversary Malt\r\n(45,1%, Whisky-Doris ...",coastal,0.00
2,2,"11th Anniversary Malt\r\n(45,1%, Whisky-Doris ...",feinty,0.00
3,3,"11th Anniversary Malt\r\n(45,1%, Whisky-Doris ...",floral,0.00
4,4,"11th Anniversary Malt\r\n(45,1%, Whisky-Doris ...",fruity,0.65
...,...,...,...,...
64219,64231,"“1836” 1970 (53,5%, Malts of Scotland ‘Angel’...",roasted,0.00
64220,64232,"“1836” 1970 (53,5%, Malts of Scotland ‘Angel’...",spicy,0.04
64221,64233,"“1836” 1970 (53,5%, Malts of Scotland ‘Angel’...",sulphury,0.00
64222,64234,"“1836” 1970 (53,5%, Malts of Scotland ‘Angel’...",sweet,0.17


In [105]:
df_tasting_notes_group[df_tasting_notes_group['perc_tasting_notes_group'].isna()]

,index,review_title,tasting_notes_group,perc_tasting_notes_group


In [102]:
df_tasting_notes_group.to_csv('flavour_wheel_long.csv', index=False)

In [103]:
flavour_wheel_values = (
  df_tasting_notes_group
  .groupby('review_title')['perc_tasting_notes_group']
  .apply(list)
  .reset_index(name='flavour_wheel_values')
)
flavour_wheel_values

,review_title,flavour_wheel_values
0,"11th Anniversary Malt\r\n(45,1%, Whisky-Doris ...","[0.0, 0.0, 0.0, 0.0, 0.65, 0.0, 0.0, 0.0, 0.15..."
1,"4th Confidential Cask, 20 yo 1993 (50,3%, Kint...","[0.04, 0.0, 0.0, 0.04, 0.33, 0.04, 0.0, 0.0, 0..."
2,"A Fine Christmas Malt 16 yo (53,2%, The Whisky...","[0.03, 0.0, 0.12, 0.03, 0.31, 0.06, 0.09, 0.06..."
3,"A Fine Christmas Malt 19 yo (44,5%, The Whisky...","[0.09, 0.0, 0.03, 0.03, 0.38, 0.03, 0.03, 0.03..."
4,"A Good Old-Fashioned Christmas Whisky (50,5%, ...","[0.11, 0.0, 0.0, 0.0, 0.3, 0.04, 0.04, 0.0, 0...."
...,...,...
5347,"‘The Marketing Malt’ 15 yo 2008 (57,1%, Decade...","[0.09, 0.0, 0.09, 0.04, 0.3, 0.09, 0.0, 0.09, ..."
5348,"‘The Nameless One’ 18 yo 1995 (46,8%, The Whis...","[0.05, 0.0, 0.0, 0.05, 0.68, 0.0, 0.0, 0.0, 0...."
5349,"‘The Trojan’ 25 yo 1990\r\n(57,1%, Exile Casks...","[0.08, 0.0, 0.04, 0.04, 0.52, 0.0, 0.0, 0.0, 0..."
5350,"‘The Wanderer’ 24 yo 1992\r\n(54,6%, Exile Cas...","[0.05, 0.0, 0.1, 0.05, 0.48, 0.0, 0.0, 0.0, 0...."


In [106]:
flavour_wheel_values.to_parquet("flavour_wheel.parquet", index=False)

In [107]:
df = df.merge(df_tasting_notes_group, how='left', on=['review_title', 'tasting_notes_group'])
df = df.drop('has_tasting_note', axis=1)
df

,review_title,review_text_vector,tasting_note,tasting_notes_group,index,perc_tasting_notes_group
0,"A Secret Highland Distillery 14 yo 2010 (54,1%...","[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, ...",almond,nutty,101.0,0.04
1,"A Secret Highland Distillery 14 yo 2010 (54,1%...","[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, ...",anise,spicy,104.0,0.12
2,"A Secret Highland Distillery 14 yo 2010 (54,1%...","[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, ...",apple,fruity,100.0,0.33
3,"A Secret Highland Distillery 14 yo 2010 (54,1%...","[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, ...",apricot,fruity,100.0,0.33
4,"A Secret Highland Distillery 14 yo 2010 (54,1%...","[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, ...",ash,peaty,102.0,0.00
...,...,...,...,...,...,...
638787,"The Balvenie 30y ‘Thirty’ (47,3%, OB 2004)","[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...",warm,spicy,59540.0,0.18
638788,"The Balvenie 30y ‘Thirty’ (47,3%, OB 2004)","[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...",wax,feinty,59534.0,0.00
638789,"The Balvenie 30y ‘Thirty’ (47,3%, OB 2004)","[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...",wine,fruity,59536.0,0.41
638790,"The Balvenie 30y ‘Thirty’ (47,3%, OB 2004)","[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...",wood,woody,59543.0,0.06


In [108]:
df[df['tasting_notes_group'].isna()]['tasting_note'].unique()

array([], dtype=object)

In [109]:
freq = review_text_vec.sum(axis=0)

freq_df = pd.DataFrame({
    'word': counter.get_feature_names_out(),
    'frequency': freq
}).sort_values(by='frequency', ascending=False).reset_index(drop=True)

print(freq_df)

              word  frequency
0            fruit       3838
1            sweet       3311
2              oak       2732
3           pepper       2665
4            spice       2597
..             ...        ...
107     gooseberry        304
108         iodine        296
109       hazelnut        286
110          mocha        258
111  passion fruit          0

[112 rows x 2 columns]


In [110]:
words_freq = (review_text_vec.T @ review_text_vec)

df_words_freq = pd.DataFrame(words_freq, columns=tasting_notes_words_filtered, index=tasting_notes_words_filtered)
np.fill_diagonal(df_words_freq.values, -1)

df_words_freq

,almond,anise,apple,apricot,ash,banana,barley,berry,bitter,blackberry,...,toffee,tropic,vanilla,varnish,walnut,warm,wax,wine,wood,zest
almond,-1,60,413,176,95,204,71,87,216,29,...,163,117,504,41,155,130,243,52,301,132
anise,60,-1,193,66,37,113,43,59,95,21,...,59,56,213,31,70,57,92,27,183,73
apple,413,193,-1,329,132,471,252,246,463,78,...,390,228,1179,91,290,243,556,124,727,395
apricot,176,66,329,-1,30,215,38,132,177,29,...,140,198,490,54,138,131,279,56,374,104
ash,95,37,132,30,-1,27,24,42,85,19,...,67,44,177,5,76,103,63,20,124,59
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
warm,130,57,243,131,103,133,47,70,106,24,...,105,98,343,17,94,-1,185,33,219,81
wax,243,92,556,279,63,314,92,85,270,22,...,105,248,548,32,167,185,-1,36,334,253
wine,52,27,124,56,20,46,19,112,91,43,...,82,32,119,4,69,33,36,-1,129,40
wood,301,183,727,374,124,441,99,339,491,156,...,370,247,920,182,395,219,334,129,-1,219
